<a href="https://colab.research.google.com/github/cliche-niche/model-zoo-submissions/blob/main/res2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from keras.datasets import cifar10
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import activations

In [13]:
(trainIm, trainLab), (testIm, testLab) = cifar10.load_data()
trainIm = trainIm / 255.0
testIm = testIm / 255.0

In [14]:
#For callbacks
class mcb(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
       if(logs.get('accuracy')>0.99):
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True
cb = mcb()

https://www.tensorflow.org/guide/keras/custom_layers_and_models

Two classes, `resId` and `resSh`, with reference from the link above were written.

`resId` is used to make a resNet block in which Identity shortcuts are used, padding is used to keep dimensions of residual the same as input (inp). They are then added together and a LeakyReLU layer is applied on them

`resSh` is used to make a resNet block in which Shortcuts are used, the size of input is made the same as residual by using a 1x1 convolution with `strides= (2, 2)`. The residual is obtained by first using a `3x3 convolution` (with padding) and `strides= (2, 2)`, followed by another `3x3 convolution` (with padding) but `strides=1` this time.

A layer for `BatchNormalization`, followed by a `LeakyReLU` is used after every convolution, unless they're being added, in which case `LeakyReLU` is applied after their addition

In [15]:
class resId(layers.Layer):
    def __init__(self, filters=64, kerSize=3):
        super(resId, self).__init__()
        self.co = layers.Conv2D(filters, kernel_size=kerSize, padding='same')
        self.bn = layers.BatchNormalization()
        self.re = layers.LeakyReLU()
    def call(self, inp):
        x = inp
        id = inp
        x = self.co(x)
        x = self.bn(x)
        x = self.re(x)
        x = self.co(x)
        x = self.bn(x)
        return self.re(x+id)

class resSh(layers.Layer):
    def __init__(self, filters, kerSize=3, stride=(2, 2)):
        super(resSh, self).__init__()
        self.co1 = layers.Conv2D(filters, kernel_size=kerSize, padding='same', strides=stride)
        self.bn = layers.BatchNormalization()
        self.re = layers.LeakyReLU()
        self.co2 = layers.Conv2D(filters, kernel_size=kerSize, padding='same')
        self.co = layers.Conv2D(filters, (1, 1), strides=stride)
    def call(self, inp):
        x = inp
        sh = inp
        x = self.co1(x)
        x = self.bn(x)
        x = self.re(x)
        x = self.co2(x)
        x = self.bn(x)
        sh = self.co(sh)
        sh = self.bn(sh)
        return self.re(x+sh)

This is a resNet model based on the paper:
https://arxiv.org/abs/1512.03385

It has some additional `Dropout` layers to obtain a better accuracy on test data, rather than without the layers.

In [16]:
class resNet(tf.keras.Model):
    def __init__(self):
        super(resNet, self).__init__()
        self.co1 = layers.Conv2D(64, (7, 7), strides=(2, 2), padding='same')
        self.mp1 = layers.MaxPooling2D(2, 2)

        self.ri1 = resId(filters=64, kerSize=3)
        self.rs1 = resSh(filters=128, kerSize=3, stride=(2, 2))
        self.ri2 = resId(filters=128, kerSize=3)
        self.rs2 = resSh(filters=256, kerSize=3, stride=(2, 2))
        self.ri3 = resId(filters=256, kerSize=3)
        self.rs3 = resSh(filters=512, kerSize=3, stride=(2, 2))
        self.ri4 = resId(filters=512, kerSize=3)

        self.gap = layers.GlobalAveragePooling2D()
        self.end = layers.Dense(10, activation="softmax")

    def call(self, inp):
        x = inp
        x = self.co1(x)
        x = self.mp1(x)
        for i in range(3):
            x = self.ri1(x)
        x = self.rs1(x)
        x = layers.Dropout(0.25)(x)
        for i in range(3):
            x = self.ri2(x)
        x = self.rs2(x)
        x = layers.Dropout(0.25)(x)
        for i in range(5):
            x = self.ri3(x)
        x = self.rs3(x)
        x = layers.Dropout(0.2)(x)
        x = self.ri4(x)
        x = self.ri4(x)

        x = self.gap(x)
        x = self.end(x)
        return(x)


In [17]:
model = resNet()
model.compile(  optimizer = tf.keras.optimizers.Adam() ,
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])

In [18]:
model.fit(trainIm, trainLab, epochs=100, callbacks=[cb])

Epoch 1/100
1563/1563 [==============================] - 54s 33ms/step - loss: 1.8192 - accuracy: 0.3764
Epoch 2/100
1563/1563 [==============================] - 51s 33ms/step - loss: 1.4478 - accuracy: 0.4963
Epoch 3/100
1563/1563 [==============================] - 51s 33ms/step - loss: 1.2691 - accuracy: 0.5597
Epoch 4/100
1563/1563 [==============================] - 51s 33ms/step - loss: 1.1570 - accuracy: 0.6017
Epoch 5/100
1563/1563 [==============================] - 51s 33ms/step - loss: 1.0853 - accuracy: 0.6265
Epoch 6/100
1563/1563 [==============================] - 51s 33ms/step - loss: 0.9890 - accuracy: 0.6602
Epoch 7/100
1563/1563 [==============================] - 51s 33ms/step - loss: 0.9525 - accuracy: 0.6741
Epoch 8/100
1563/1563 [==============================] - 51s 33ms/step - loss: 0.9182 - accuracy: 0.6875
Epoch 9/100
1563/1563 [==============================] - 51s 33ms/step - loss: 0.8264 - accuracy: 0.7177
Epoch 10/100
1563/1563 [==============================]

In [20]:
model.evaluate(testIm, testLab)

313/313 [==============================] - 4s 12ms/step - loss: 2.5060 - accuracy: 0.1987


[2.505959987640381, 0.19869999587535858]